In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from nltk import PorterStemmer
import re

In [2]:
# read attribute.csv file
attribute_df = pd.read_csv("attributes.csv")

In [3]:
attribute_df.head()

product_uid      name                                              value
0     100001.0  Bullet01  Versatile connector for various 90° connection...
1     100001.0  Bullet02  Stronger than angled nailing or screw fastenin...
2     100001.0  Bullet03  Help ensure joints are consistently straight a...
3     100001.0  Bullet04              Dimensions: 3 in. x 3 in. x 1-1/2 in.
4     100001.0  Bullet05                           Made from 12-Gauge steel

In [4]:
attribute_df.isnull().sum()

product_uid     155
name            155
value          2284
dtype: int64

In [5]:
# drop null values rows
attribute_df = attribute_df.dropna()

In [6]:
# merge name's multiple rows to single row based on product id
attribute_df_new = attribute_df.groupby('product_uid')['name'].apply(','.join).reset_index()

In [7]:
attribute_df_new.head()

product_uid                                               name
0     100001.0  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...
1     100002.0  Application Method,Assembled Depth (in.),Assem...
2     100003.0  Built-in flange,Bullet01,Bullet02,Bullet03,Bul...
3     100004.0  Amperage (amps),Bullet01,Bullet02,Bullet03,Bul...
4     100005.0  Bath Faucet Type,Built-in Water Filter,Bullet0...

In [8]:
# product_uid is float, convert it into int
attribute_df_new['product_uid'] = attribute_df_new['product_uid'].astype(int)

In [9]:
# value is object type, convert it into string
attribute_df['value'] = attribute_df['value'].astype(str)

In [10]:
#create a new df to store the merged values
attribute_df_new1 = attribute_df.groupby('product_uid')['value'].apply(' '.join).reset_index()

In [11]:
attribute_df_new1['product_uid'] = attribute_df_new1['product_uid'].astype(int)

In [12]:
attribute_df_new1.head()

product_uid                                              value
0       100001  Versatile connector for various 90° connection...
1       100002  Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...
2       100003  Yes Slightly narrower for tighter spaces Desig...
3       100004  8.56 Positive power tolerance (0 to +5-Watt) A...
4       100005  Combo Tub and Shower No Includes the trim kit ...

In [13]:
# join the two df
attribute_df_merged = pd.merge(left=attribute_df_new, right=attribute_df_new1, how='left', left_on='product_uid', right_on='product_uid')

In [14]:
attribute_df_merged.head()

product_uid                                               name  \
0       100001  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...   
1       100002  Application Method,Assembled Depth (in.),Assem...   
2       100003  Built-in flange,Bullet01,Bullet02,Bullet03,Bul...   
3       100004  Amperage (amps),Bullet01,Bullet02,Bullet03,Bul...   
4       100005  Bath Faucet Type,Built-in Water Filter,Bullet0...   

                                               value  
0  Versatile connector for various 90° connection...  
1  Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...  
2  Yes Slightly narrower for tighter spaces Desig...  
3  8.56 Positive power tolerance (0 to +5-Watt) A...  
4  Combo Tub and Shower No Includes the trim kit ...

In [15]:
# read the product description csv file
df_prod_description = pd.read_csv("product_descriptions.csv", encoding='ISO-8859-1')

In [16]:
df_prod_description.head()

product_uid                                product_description
0       100001  Not only do angles make joints stronger, they ...
1       100002  BEHR Premium Textured DECKOVER is an innovativ...
2       100003  Classic architecture meets contemporary design...
3       100004  The Grape Solar 265-Watt Polycrystalline PV So...
4       100005  Update your bathroom with the Delta Vero Singl...

In [17]:
df_prod_description.isnull().sum()

product_uid            0
product_description    0
dtype: int64

In [18]:
# drop the nan values
df_prod_description = df_prod_description.dropna()

In [19]:
# merge the attribute table with product description
attribute_df_merged = pd.merge(left=attribute_df_merged, right=df_prod_description, how='left', left_on='product_uid', right_on='product_uid')

In [20]:
attribute_df_merged.head()

product_uid                                               name  \
0       100001  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...   
1       100002  Application Method,Assembled Depth (in.),Assem...   
2       100003  Built-in flange,Bullet01,Bullet02,Bullet03,Bul...   
3       100004  Amperage (amps),Bullet01,Bullet02,Bullet03,Bul...   
4       100005  Bath Faucet Type,Built-in Water Filter,Bullet0...   

                                               value  \
0  Versatile connector for various 90° connection...   
1  Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...   
2  Yes Slightly narrower for tighter spaces Desig...   
3  8.56 Positive power tolerance (0 to +5-Watt) A...   
4  Combo Tub and Shower No Includes the trim kit ...   

                                 product_description  
0  Not only do angles make joints stronger, they ...  
1  BEHR Premium Textured DECKOVER is an innovativ...  
2  Classic architecture meets contemporary design...  
3  The Grape Solar 265-Watt Polycrystalline PV So...  
4  Update your bathroom with the Delta Vero Singl...

In [21]:
train_df = pd.read_csv("train.csv", encoding='ISO-8859-1')

In [22]:
train_df.head()

id  product_uid                                      product_title  \
0   2       100001                  Simpson Strong-Tie 12-Gauge Angle   
1   3       100001                  Simpson Strong-Tie 12-Gauge Angle   
2   9       100002  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...   
3  16       100005  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   
4  17       100005  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   

          search_term  relevance  
0       angle bracket       3.00  
1           l bracket       2.50  
2           deck over       3.00  
3    rain shower head       2.33  
4  shower only faucet       2.67

In [23]:
train_df.isnull().sum()

id               0
product_uid      0
product_title    0
search_term      0
relevance        0
dtype: int64

In [24]:
# merge the search_term 
df1_train_df = train_df.groupby('product_uid')['search_term'].apply(' ,'.join).reset_index()

In [25]:
df1_train_df.head()

product_uid                                       search_term
0       100001                          angle bracket ,l bracket
1       100002                                         deck over
2       100005              rain shower head ,shower only faucet
3       100006  convection otr ,microwave over stove ,microwaves
4       100007                                   emergency light

In [26]:
train_df['relevance'] = train_df['relevance'].astype('str')

In [27]:
df2_train_df = train_df.groupby('product_uid')['relevance'].apply(','.join).reset_index()

In [28]:
df2_train_df.head()

product_uid     relevance
0       100001       3.0,2.5
1       100002           3.0
2       100005     2.33,2.67
3       100006  3.0,2.67,3.0
4       100007          2.67

In [29]:
df3_train_df = train_df.groupby('product_uid')['product_title'].apply(','.join).reset_index()

In [30]:
df3_train_df.head()

product_uid                                      product_title
0       100001  Simpson Strong-Tie 12-Gauge Angle,Simpson Stro...
1       100002  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...
2       100005  Delta Vero 1-Handle Shower Only Faucet Trim Ki...
3       100006  Whirlpool 1.9 cu. ft. Over the Range Convectio...
4       100007  Lithonia Lighting Quantum 2-Light Black LED Em...

In [31]:
merged_train = pd.merge(left = df1_train_df, right = df2_train_df, how = 'left', left_on = 'product_uid', right_on='product_uid')

In [32]:
merged_train.head()

product_uid                                       search_term     relevance
0       100001                          angle bracket ,l bracket       3.0,2.5
1       100002                                         deck over           3.0
2       100005              rain shower head ,shower only faucet     2.33,2.67
3       100006  convection otr ,microwave over stove ,microwaves  3.0,2.67,3.0
4       100007                                   emergency light          2.67

In [33]:
merged_train = pd.merge(left = merged_train, right = df3_train_df, how = 'left', left_on = 'product_uid', right_on='product_uid')

In [34]:
merged_train.head()

product_uid                                       search_term  \
0       100001                          angle bracket ,l bracket   
1       100002                                         deck over   
2       100005              rain shower head ,shower only faucet   
3       100006  convection otr ,microwave over stove ,microwaves   
4       100007                                   emergency light   

      relevance                                      product_title  
0       3.0,2.5  Simpson Strong-Tie 12-Gauge Angle,Simpson Stro...  
1           3.0  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...  
2     2.33,2.67  Delta Vero 1-Handle Shower Only Faucet Trim Ki...  
3  3.0,2.67,3.0  Whirlpool 1.9 cu. ft. Over the Range Convectio...  
4          2.67  Lithonia Lighting Quantum 2-Light Black LED Em...

In [35]:
attribute_df_merged.head()

product_uid                                               name  \
0       100001  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...   
1       100002  Application Method,Assembled Depth (in.),Assem...   
2       100003  Built-in flange,Bullet01,Bullet02,Bullet03,Bul...   
3       100004  Amperage (amps),Bullet01,Bullet02,Bullet03,Bul...   
4       100005  Bath Faucet Type,Built-in Water Filter,Bullet0...   

                                               value  \
0  Versatile connector for various 90° connection...   
1  Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...   
2  Yes Slightly narrower for tighter spaces Desig...   
3  8.56 Positive power tolerance (0 to +5-Watt) A...   
4  Combo Tub and Shower No Includes the trim kit ...   

                                 product_description  
0  Not only do angles make joints stronger, they ...  
1  BEHR Premium Textured DECKOVER is an innovativ...  
2  Classic architecture meets contemporary design...  
3  The Grape Solar 265-Watt Polycrystalline PV So...  
4  Update your bathroom with the Delta Vero Singl...

In [36]:
result_df = pd.merge(left=merged_train, right=attribute_df_merged, how='outer', left_on='product_uid', right_on='product_uid')

In [37]:
result_df.head()

product_uid                                       search_term  \
0       100001                          angle bracket ,l bracket   
1       100002                                         deck over   
2       100005              rain shower head ,shower only faucet   
3       100006  convection otr ,microwave over stove ,microwaves   
4       100007                                   emergency light   

      relevance                                      product_title  \
0       3.0,2.5  Simpson Strong-Tie 12-Gauge Angle,Simpson Stro...   
1           3.0  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...   
2     2.33,2.67  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   
3  3.0,2.67,3.0  Whirlpool 1.9 cu. ft. Over the Range Convectio...   
4          2.67  Lithonia Lighting Quantum 2-Light Black LED Em...   

                                                name  \
0  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...   
1  Application Method,Assembled Depth (in.),Assem...   
2  Bath Faucet Type,Built-in Water Filter,Bullet0...   
3  Appliance Type,Assembled Depth (in.),Assembled...   
4  Battery Power Type,Battery Size,Bulb Type Incl...   

                                               value  \
0  Versatile connector for various 90° connection...   
1  Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...   
2  Combo Tub and Shower No Includes the trim kit ...   
3  Over the Range Microwave 18.5 in 17.13 in 29.9...   
4  Ni-Cad .Built-In LED Advanced LED technology i...   

                                 product_description  
0  Not only do angles make joints stronger, they ...  
1  BEHR Premium Textured DECKOVER is an innovativ...  
2  Update your bathroom with the Delta Vero Singl...  
3  Achieving delicious results is almost effortle...  
4  The Quantum Adjustable 2-Light LED Black Emerg...

In [38]:
result_df.shape

(102526, 7)

In [39]:
result_df = result_df.dropna()

In [40]:
result_df.isnull().count()

product_uid            38404
search_term            38404
relevance              38404
product_title          38404
name                   38404
value                  38404
product_description    38404
dtype: int64

In [41]:
result_df.shape

(38404, 7)

In [42]:
result_df.isnull().sum()

product_uid            0
search_term            0
relevance              0
product_title          0
name                   0
value                  0
product_description    0
dtype: int64

In [43]:
result_df.head()

product_uid                                       search_term  \
0       100001                          angle bracket ,l bracket   
1       100002                                         deck over   
2       100005              rain shower head ,shower only faucet   
3       100006  convection otr ,microwave over stove ,microwaves   
4       100007                                   emergency light   

      relevance                                      product_title  \
0       3.0,2.5  Simpson Strong-Tie 12-Gauge Angle,Simpson Stro...   
1           3.0  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...   
2     2.33,2.67  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   
3  3.0,2.67,3.0  Whirlpool 1.9 cu. ft. Over the Range Convectio...   
4          2.67  Lithonia Lighting Quantum 2-Light Black LED Em...   

                                                name  \
0  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...   
1  Application Method,Assembled Depth (in.),Assem...   
2  Bath Faucet Type,Built-in Water Filter,Bullet0...   
3  Appliance Type,Assembled Depth (in.),Assembled...   
4  Battery Power Type,Battery Size,Bulb Type Incl...   

                                               value  \
0  Versatile connector for various 90° connection...   
1  Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...   
2  Combo Tub and Shower No Includes the trim kit ...   
3  Over the Range Microwave 18.5 in 17.13 in 29.9...   
4  Ni-Cad .Built-In LED Advanced LED technology i...   

                                 product_description  
0  Not only do angles make joints stronger, they ...  
1  BEHR Premium Textured DECKOVER is an innovativ...  
2  Update your bathroom with the Delta Vero Singl...  
3  Achieving delicious results is almost effortle...  
4  The Quantum Adjustable 2-Light LED Black Emerg...

In [44]:
result_df.to_csv("IR_Data")

In [45]:
result_df.drop('relevance', inplace=True, axis=1)

In [46]:
result_df.head()

product_uid                                       search_term  \
0       100001                          angle bracket ,l bracket   
1       100002                                         deck over   
2       100005              rain shower head ,shower only faucet   
3       100006  convection otr ,microwave over stove ,microwaves   
4       100007                                   emergency light   

                                       product_title  \
0  Simpson Strong-Tie 12-Gauge Angle,Simpson Stro...   
1  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...   
2  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   
3  Whirlpool 1.9 cu. ft. Over the Range Convectio...   
4  Lithonia Lighting Quantum 2-Light Black LED Em...   

                                                name  \
0  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...   
1  Application Method,Assembled Depth (in.),Assem...   
2  Bath Faucet Type,Built-in Water Filter,Bullet0...   
3  Appliance Type,Assembled Depth (in.),Assembled...   
4  Battery Power Type,Battery Size,Bulb Type Incl...   

                                               value  \
0  Versatile connector for various 90° connection...   
1  Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...   
2  Combo Tub and Shower No Includes the trim kit ...   
3  Over the Range Microwave 18.5 in 17.13 in 29.9...   
4  Ni-Cad .Built-In LED Advanced LED technology i...   

                                 product_description  
0  Not only do angles make joints stronger, they ...  
1  BEHR Premium Textured DECKOVER is an innovativ...  
2  Update your bathroom with the Delta Vero Singl...  
3  Achieving delicious results is almost effortle...  
4  The Quantum Adjustable 2-Light LED Black Emerg...

In [47]:
result_df.to_csv("IR_Data_without_relevance")

In [48]:
result_df["bag_of_words"] = result_df["name"] + result_df["value"] + result_df["product_description"]

In [49]:
result_df = result_df.drop(["name","value","product_description"], axis = 1)

In [50]:
result_df.head()

product_uid                                       search_term  \
0       100001                          angle bracket ,l bracket   
1       100002                                         deck over   
2       100005              rain shower head ,shower only faucet   
3       100006  convection otr ,microwave over stove ,microwaves   
4       100007                                   emergency light   

                                       product_title  \
0  Simpson Strong-Tie 12-Gauge Angle,Simpson Stro...   
1  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...   
2  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   
3  Whirlpool 1.9 cu. ft. Over the Range Convectio...   
4  Lithonia Lighting Quantum 2-Light Black LED Em...   

                                        bag_of_words  
0  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...  
1  Application Method,Assembled Depth (in.),Assem...  
2  Bath Faucet Type,Built-in Water Filter,Bullet0...  
3  Appliance Type,Assembled Depth (in.),Assembled...  
4  Battery Power Type,Battery Size,Bulb Type Incl...

In [51]:
result_df["bag_of_words_cleaned"] = result_df["bag_of_words"].apply(lambda words: ' '.join(word.strip().lower() for word in words.split(",") if word not in stop))

In [52]:
# remove the regular expression
for index, row in result_df.iterrows():
    result_df.loc[index,'bag_of_words_cleaned'] = (re.sub('[^a-zA-Z0-9 \n]', '', row['bag_of_words_cleaned']))
    

In [53]:
result_df.head()

product_uid                                       search_term  \
0       100001                          angle bracket ,l bracket   
1       100002                                         deck over   
2       100005              rain shower head ,shower only faucet   
3       100006  convection otr ,microwave over stove ,microwaves   
4       100007                                   emergency light   

                                       product_title  \
0  Simpson Strong-Tie 12-Gauge Angle,Simpson Stro...   
1  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...   
2  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   
3  Whirlpool 1.9 cu. ft. Over the Range Convectio...   
4  Lithonia Lighting Quantum 2-Light Black LED Em...   

                                        bag_of_words  \
0  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...   
1  Application Method,Assembled Depth (in.),Assem...   
2  Bath Faucet Type,Built-in Water Filter,Bullet0...   
3  Appliance Type,Assembled Depth (in.),Assembled...   
4  Battery Power Type,Battery Size,Bulb Type Incl...   

                                bag_of_words_cleaned  
0  bullet01 bullet02 bullet03 bullet04 bullet05 b...  
1  application method assembled depth in assemble...  
2  bath faucet type builtin water filter bullet01...  
3  appliance type assembled depth in assembled he...  
4  battery power type battery size bulb type incl...

In [54]:
result_df['bag_of_words_cleaned'].head()

0    bullet01 bullet02 bullet03 bullet04 bullet05 b...
1    application method assembled depth in assemble...
2    bath faucet type builtin water filter bullet01...
3    appliance type assembled depth in assembled he...
4    battery power type battery size bulb type incl...
Name: bag_of_words_cleaned, dtype: object

In [55]:
# instansiate the RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [56]:
# change the bag of words to string
result_df['bag_of_words_cleaned'] = result_df['bag_of_words_cleaned'].astype("str")

In [57]:
result_df['bag_of_words_cleaned'] = result_df['bag_of_words_cleaned'].replace("in", "")

In [58]:
# tokenize the string
result_df['bag_of_words_cleaned'] = result_df['bag_of_words_cleaned'].apply(lambda x: tokenizer.tokenize(x))

In [59]:
stop_words = set(stopwords.words('english')) 
def remove_stopwords(text):
    
    words = [w for w in text if w not in stop_words]
    return words

In [60]:
result_df['bag_of_words_cleaned'] = result_df['bag_of_words_cleaned'].apply(lambda x : remove_stopwords(x))
result_df['bag_of_words_cleaned'].head()

0    [bullet01, bullet02, bullet03, bullet04, bulle...
1    [application, method, assembled, depth, assemb...
2    [bath, faucet, type, builtin, water, filter, b...
3    [appliance, type, assembled, depth, assembled,...
4    [battery, power, type, battery, size, bulb, ty...
Name: bag_of_words_cleaned, dtype: object

### lemmatization

In [61]:
# instantiate lemmatizer
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

In [62]:
result_df['bag_of_words_cleaned'] = result_df['bag_of_words_cleaned'].apply(lambda x: word_lemmatizer(x))

### stemming

In [63]:
# instantiate stemmer
stemmer = PorterStemmer()

In [64]:
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text


In [65]:
result_df['bag_of_words_cleaned'] = result_df['bag_of_words_cleaned'].apply(lambda x: word_stemmer(x))

In [66]:
result_df['bag_of_words_cleaned'] = result_df['bag_of_words_cleaned'].astype(str)

In [67]:
import collections
count = collections.Counter(result_df['bag_of_words_cleaned'][:1])

In [68]:
from nltk.tokenize import word_tokenize

processed_text = []

for i in result_df['bag_of_words_cleaned']:
    text = i.strip()
    processed_text.append(word_tokenize(str(text)))
    

In [69]:
total_vocab = [x for x in result_df]

In [70]:
result_df.head()

product_uid                                       search_term  \
0       100001                          angle bracket ,l bracket   
1       100002                                         deck over   
2       100005              rain shower head ,shower only faucet   
3       100006  convection otr ,microwave over stove ,microwaves   
4       100007                                   emergency light   

                                       product_title  \
0  Simpson Strong-Tie 12-Gauge Angle,Simpson Stro...   
1  BEHR Premium Textured DeckOver 1-gal. #SC-141 ...   
2  Delta Vero 1-Handle Shower Only Faucet Trim Ki...   
3  Whirlpool 1.9 cu. ft. Over the Range Convectio...   
4  Lithonia Lighting Quantum 2-Light Black LED Em...   

                                        bag_of_words  \
0  Bullet01,Bullet02,Bullet03,Bullet04,Bullet05,B...   
1  Application Method,Assembled Depth (in.),Assem...   
2  Bath Faucet Type,Built-in Water Filter,Bullet0...   
3  Appliance Type,Assembled Depth (in.),Assembled...   
4  Battery Power Type,Battery Size,Bulb Type Incl...   

                                bag_of_words_cleaned  
0  bullet01 bullet02 bullet03 bullet04 bullet05 b...  
1  applic method assembl depth assembl height ass...  
2  bath faucet type builtin water filter bullet01...  
3  applianc type assembl depth assembl height ass...  
4  batteri power type batteri size bulb type incl...

In [71]:
cleaned_df = result_df.drop(['bag_of_words'], axis = 1)

In [72]:
cleaned_df.to_csv("cleaned_IR_data_final")